In [ ]:
import geopandas as gpd
from geopandas.geoseries import *

# 447 PLRs
file_path_of_plr = 'berlin-lor.planungsraeume.geojson'
plr = gpd.read_file(file_path_of_plr)

plr.insert(0, 'New_ID', range(0, 0 + len(plr)))
plr.to_file('output_berlin-lor.planungsraeume.geojson', driver='GeoJSON')

projection = "EPSG:5678" # https://epsg.io/5678
# buffer in meter (3 km/h walking pace; what is reachable in 10 min if your start at the boundary of PLR) => 500 m
bufferSize = 500 
plr = plr.to_crs(projection)

# 11522 forests
file_path_of_forests = 'waldflaechen.geojson'
forests = gpd.read_file(file_path_of_forests)
forests.insert(0, 'New_ID', range(0, 0 + len(forests)))
forests.to_file("output_waldflaechen.geojson", driver="GeoJSON")
forests = forests.to_crs(projection)

forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

# calc area 
plr['area'] = None
plr['bufferedGeometry'] = None
plr['bufferedArea'] = None

plr['bufferedGeometry'] = plr.geometry.buffer(bufferSize)
plr['bufferedArea'] = None

for index, row in plr.iterrows():
    plr.loc[index, 'area'] = row['geometry'].area / 10000
    plr.loc[index, 'bufferedArea'] = row['bufferedGeometry'].area / 10000

# add open space to calc which of them intersects with bufferedGeometry 
file_path_of_open_spaces = 'gruenflaechen_shapes.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)
open_spaces.insert(0, 'New_ID', range(len(forests), len(forests) + len(open_spaces)))
open_spaces.to_file('output_gruenflaechen_shapes.geojson', driver='GeoJSON')
# 2509 open spaces
open_spaces = open_spaces.to_crs(projection)

plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesArea'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesBufferedArea'] = np.empty((len(plr), 0)).tolist()

# print(open_spaces['KATASTERFL'].sum()) # => 61376300.8 => 6137.63008 hectar

for indexPLR, rowPLR in plr.iterrows():
    for indexOS, rowOS in open_spaces.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(rowOS.New_ID)
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowOS.KATASTERFL)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(rowOS.New_ID)
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowOS.KATASTERFL)
        if rowPLR.SCHLUESSEL.startswith('0'):
            plr.loc[indexPLR, 'SCHLUESSEL'] = rowPLR.SCHLUESSEL[1:]
    for indexF, rowF in forests.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(rowF.New_ID)
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowF.area_text)
        if rowPLR.geometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(rowF.New_ID)
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowF.area_text)

newplr = plr

plr.head()

# # sum of all PLRs => 89132.31203627815 hectar
# sumOS = plr['area'].sum()
# print(sumOS)

newplr.astype({'geometry': str}, {'bufferedGeometry': str}).to_csv('PLR_buffered.csv', sep=';')

In [ ]:
import pandas as pd 
import numpy as np

newplr['numberOfSpacesInPLR'] = None
newplr['numberOfSpacesInBufferedPLR'] = None

for index, row in newplr.iterrows():
    newplr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    newplr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)

data = pd.read_csv('EWR201812E_Matrix.csv', sep=';')
data.rename(columns = {'RAUMID':'SCHLUESSEL'}, inplace = True)
data = data[['SCHLUESSEL', 'E_E']]

data['SCHLUESSEL'] = data['SCHLUESSEL'].astype(str)
newplr['SCHLUESSEL'] = newplr['SCHLUESSEL'].astype(str)

output = pd.merge(newplr, data, on='SCHLUESSEL')
output = output.rename(columns={'E_E': 'population'})

output['openSpacesPerInhabitant'] = None

for index, row in output.iterrows():
    output.loc[index, 'allOpenSpacesBufferedArea'] = sum(row.allOpenSpacesBufferedArea) / 10000
    output.loc[index, 'allOpenSpacesArea'] = sum(row.allOpenSpacesArea) / 10000
    output.loc[index, 'openSpacesPerInhabitant'] = output.loc[index, 'allOpenSpacesBufferedArea'] * 10000 / row.population    

output = output.to_crs('EPSG:4326')

output.astype({'geometry': str}).astype({'bufferedGeometry': str}).to_csv('data.csv', index=False, header=True)

#output = output.to_json(orient='records')

#output.head()

print(output['population'].sum())

# Als Einwohner von PLR x hat man y Grünflächen mit z m^2 Fläche zur Verfügung 
# check Einheiten
# Gesamtwaldflächen berechnen und vergleichen
# json string ['400']
# remove NaNs
# get PLZ, Bezirk for PLRs
# Wer hat zuwenig GF?
# Grünfläche in Grünfläche?